In [ ]:
# package imports...

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# dataset CSV imports...

In [2]:
df_maths = pd.read_csv('student_mat.csv', delimiter = ';')
df_portuguese = pd.read_csv('student_por.csv', delimiter = ';')

In [ ]:
# functions...

In [3]:
def SORT_DF(df, column, ascending_TF):

    """ organises a dataset in ascending or
    descending order of the input column """
    
    df = df.sort_values(by = f'{column}', ascending = bool(f'{ascending_TF}'))
    return df

def CATEGORICAL_DATAPOINTS(school, 
                           sex, 
                           address, 
                           famsize, 
                           Pstatus, 
                           Mjob, 
                           Fjob, 
                           reason, 
                           guardian, 
                           schoolsup,
                           famsup,
                           paid,
                           activities,
                           nursery,
                           higher,
                           internet, 
                           romantic,):   

    categorical_dpoints = f''' the unique points contained within each column are as follows...
    
    school:
    {school}
    
    sex: 
    {sex}
    
    address: 
    {address}
    
    famsize: 
    {famsize}
    
    Pstatus: 
    {Pstatus}
    
    Mjob:
    {Mjob}
    
    Fjob:
    {Fjob}
    
    reason:
    {reason}
    
    guardian:
    {guardian}
    
    schoolsup:
    {schoolsup}
    
    famsup:
    {famsup}

    paid:
    {paid}

    activities:
    {activities}

    nursery:
    {nursery}

    higher:
    {higher}

    internet:
    {internet}

    romantic:
    {romantic}
    '''

    print(categorical_dpoints)
    return categorical_dpoints

In [ ]:
# Exploratory Data Analysis...

In [4]:
df_maths.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [5]:
df_maths.tail()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10
394,MS,M,19,U,LE3,T,1,1,other,at_home,...,3,2,3,3,3,5,5,8,9,9


In [6]:
df_maths_cat = df_maths.select_dtypes(include = 'object')

print(df_maths_cat)

    school sex address famsize Pstatus      Mjob      Fjob  reason guardian  \
0       GP   F       U     GT3       A   at_home   teacher  course   mother   
1       GP   F       U     GT3       T   at_home     other  course   father   
2       GP   F       U     LE3       T   at_home     other   other   mother   
3       GP   F       U     GT3       T    health  services    home   mother   
4       GP   F       U     GT3       T     other     other    home   father   
..     ...  ..     ...     ...     ...       ...       ...     ...      ...   
390     MS   M       U     LE3       A  services  services  course    other   
391     MS   M       U     LE3       T  services  services  course   mother   
392     MS   M       R     GT3       T     other     other  course    other   
393     MS   M       R     LE3       T  services     other  course   mother   
394     MS   M       U     LE3       T     other   at_home  course   father   

    schoolsup famsup paid activities nursery higher

In [7]:
maths_catdpoints = CATEGORICAL_DATAPOINTS(school = df_maths['school'].unique(), 
                                          sex = df_maths['sex'].unique(), 
                                          address = df_maths['address'].unique(), 
                                          famsize = df_maths['famsize'].unique(), 
                                          Pstatus = df_maths['Pstatus'].unique(), 
                                          Mjob = df_maths['Mjob'].unique(), 
                                          Fjob = df_maths['Fjob'].unique(), 
                                          reason = df_maths['reason'].unique(), 
                                          guardian = df_maths['guardian'].unique(), 
                                          schoolsup = df_maths['schoolsup'].unique(),
                                          famsup = df_maths['famsup'].unique(),
                                          paid = df_maths['paid'].unique(),
                                          activities = df_maths['activities'].unique(),
                                          nursery = df_maths['nursery'].unique(),
                                          higher = df_maths['higher'].unique(),
                                          internet = df_maths['internet'].unique(), 
                                          romantic = df_maths['romantic'].unique(),)   

 the unique points contained within each column are as follows...
    
    school:
    ['GP' 'MS']
    
    sex: 
    ['F' 'M']
    
    address: 
    ['U' 'R']
    
    famsize: 
    ['GT3' 'LE3']
    
    Pstatus: 
    ['A' 'T']
    
    Mjob:
    ['at_home' 'health' 'other' 'services' 'teacher']
    
    Fjob:
    ['teacher' 'other' 'services' 'health' 'at_home']
    
    reason:
    ['course' 'other' 'home' 'reputation']
    
    guardian:
    ['mother' 'father' 'other']
    
    schoolsup:
    ['yes' 'no']
    
    famsup:
    ['no' 'yes']

    paid:
    ['no' 'yes']

    activities:
    ['no' 'yes']

    nursery:
    ['yes' 'no']

    higher:
    ['yes' 'no']

    internet:
    ['no' 'yes']

    romantic:
    ['no' 'yes']
    


In [8]:
df_maths.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

In [9]:
df_maths.shape

(395, 33)

In [10]:
df_maths.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [11]:
df_maths.nunique()

school         2
sex            2
age            8
address        2
famsize        2
Pstatus        2
Medu           5
Fedu           5
Mjob           5
Fjob           5
reason         4
guardian       3
traveltime     4
studytime      4
failures       4
schoolsup      2
famsup         2
paid           2
activities     2
nursery        2
higher         2
internet       2
romantic       2
famrel         5
freetime       5
goout          5
Dalc           5
Walc           5
health         5
absences      34
G1            17
G2            17
G3            18
dtype: int64

In [12]:
df_maths.corr(numeric_only = True)

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
age,1.000000,-0.163658,-0.163438,0.070641,-0.004140,0.243665,0.053940,0.016434,0.126964,0.131125,0.117276,-0.062187,0.175230,-0.064081,-0.143474,-0.161579
Medu,-0.163658,1.000000,0.623455,-0.171639,0.064944,-0.236680,-0.003914,0.030891,0.064094,0.019834,-0.047123,-0.046878,0.100285,0.205341,0.215527,0.217147
Fedu,-0.163438,0.623455,1.000000,-0.158194,-0.009175,-0.250408,-0.001370,-0.012846,0.043105,0.002386,-0.012631,0.014742,0.024473,0.190270,0.164893,0.152457
traveltime,0.070641,-0.171639,-0.158194,1.000000,-0.100909,0.092239,-0.016808,-0.017025,0.028540,0.138325,0.134116,0.007501,-0.012944,-0.093040,-0.153198,-0.117142
studytime,-0.004140,0.064944,-0.009175,-0.100909,1.000000,-0.173563,0.039731,-0.143198,-0.063904,-0.196019,-0.253785,-0.075616,-0.062700,0.160612,0.135880,0.097820
failures,0.243665,-0.236680,-0.250408,0.092239,-0.173563,1.000000,-0.044337,0.091987,0.124561,0.136047,0.141962,0.065827,0.063726,-0.354718,-0.355896,-0.360415
famrel,0.053940,-0.003914,-0.001370,-0.016808,0.039731,-0.044337,1.000000,0.150701,0.064568,-0.077594,-0.113397,0.094056,-0.044354,0.022168,-0.018281,0.051363
freetime,0.016434,0.030891,-0.012846,-0.017025,-0.143198,0.091987,0.150701,1.000000,0.285019,0.209001,0.147822,0.075733,-0.058078,0.012613,-0.013777,0.011307
goout,0.126964,0.064094,0.043105,0.028540,-0.063904,0.124561,0.064568,0.285019,1.000000,0.266994,0.420386,-0.009577,0.044302,-0.149104,-0.162250,-0.132791
Dalc,0.131125,0.019834,0.002386,0.138325,-0.196019,0.136047,-0.077594,0.209001,0.266994,1.000000,0.647544,0.077180,0.111908,-0.094159,-0.064120,-0.054660


In [13]:
df_portuguese.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [14]:
df_portuguese.tail()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10
648,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,4,10,11,11


In [15]:
df_portuguese.shape

(649, 33)

In [16]:
df_portuguese.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
644     MS   F   19       R     GT3       T     2     3  services     other   
645     MS   F   18       U     LE3       T     3     1   teacher  services   
646     MS   F   18       U     GT3       T     1     1     other     other   
647     MS   M   17       U     LE3       T     3     1  services  services   
648     MS   M   18       R     LE3       T     3     2  services     other   

     ... famrel fre

In [17]:
df_portuguese.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000
mean,16.744222,2.514638,2.306626,1.568567,1.930663,0.221880,3.930663,3.180277,3.184900,1.502311,2.280431,3.536210,3.659476,11.399076,11.570108,11.906009
std,1.218138,1.134552,1.099931,0.748660,0.829510,0.593235,0.955717,1.051093,1.175766,0.924834,1.284380,1.446259,4.640759,2.745265,2.913639,3.230656
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,1.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,2.000000,0.000000,10.000000,10.000000,10.000000
50%,17.000000,2.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,2.000000,11.000000,11.000000,12.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,6.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,32.000000,19.000000,19.000000,19.000000


In [19]:
df_p_cat = df_portuguese.select_dtypes(include = 'object')

print(df_p_cat)

    school sex address famsize Pstatus      Mjob      Fjob  reason guardian  \
0       GP   F       U     GT3       A   at_home   teacher  course   mother   
1       GP   F       U     GT3       T   at_home     other  course   father   
2       GP   F       U     LE3       T   at_home     other   other   mother   
3       GP   F       U     GT3       T    health  services    home   mother   
4       GP   F       U     GT3       T     other     other    home   father   
..     ...  ..     ...     ...     ...       ...       ...     ...      ...   
644     MS   F       R     GT3       T  services     other  course   mother   
645     MS   F       U     LE3       T   teacher  services  course   mother   
646     MS   F       U     GT3       T     other     other  course   mother   
647     MS   M       U     LE3       T  services  services  course   mother   
648     MS   M       R     LE3       T  services     other  course   mother   

    schoolsup famsup paid activities nursery higher

In [20]:
portu_catdpoints = CATEGORICAL_DATAPOINTS(school = df_portuguese['school'].unique(), 
                                          sex = df_portuguese['sex'].unique(), 
                                          address = df_portuguese['address'].unique(), 
                                          famsize = df_portuguese['famsize'].unique(), 
                                          Pstatus = df_portuguese['Pstatus'].unique(), 
                                          Mjob = df_portuguese['Mjob'].unique(), 
                                          Fjob = df_portuguese['Fjob'].unique(), 
                                          reason = df_portuguese['reason'].unique(), 
                                          guardian = df_portuguese['guardian'].unique(), 
                                          schoolsup = df_portuguese['schoolsup'].unique(),
                                          famsup = df_portuguese['famsup'].unique(),
                                          paid = df_portuguese['paid'].unique(),
                                          activities = df_portuguese['activities'].unique(),
                                          nursery = df_portuguese['nursery'].unique(),
                                          higher = df_portuguese['higher'].unique(),
                                          internet = df_portuguese['internet'].unique(), 
                                          romantic = df_portuguese['romantic'].unique(),)   

 the unique points contained within each column are as follows...
    
    school:
    ['GP' 'MS']
    
    sex: 
    ['F' 'M']
    
    address: 
    ['U' 'R']
    
    famsize: 
    ['GT3' 'LE3']
    
    Pstatus: 
    ['A' 'T']
    
    Mjob:
    ['at_home' 'health' 'other' 'services' 'teacher']
    
    Fjob:
    ['teacher' 'other' 'services' 'health' 'at_home']
    
    reason:
    ['course' 'other' 'home' 'reputation']
    
    guardian:
    ['mother' 'father' 'other']
    
    schoolsup:
    ['yes' 'no']
    
    famsup:
    ['no' 'yes']

    paid:
    ['no' 'yes']

    activities:
    ['no' 'yes']

    nursery:
    ['yes' 'no']

    higher:
    ['yes' 'no']

    internet:
    ['no' 'yes']

    romantic:
    ['no' 'yes']
    
